In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint

In [2]:
aa_data = pd.read_csv('AAdatabase.csv')

In [3]:
aa_data.head()

,Year,Ceremony,Award,Winner,Name,Film
0,1927/1928,1,Actor,NaN,Richard Barthelmess,The Noose
1,1927/1928,1,Actor,1.0,Emil Jannings,The Last Command
2,1927/1928,1,Actress,NaN,Louise Dresser,A Ship Comes In
3,1927/1928,1,Actress,1.0,Janet Gaynor,7th Heaven
4,1927/1928,1,Actress,NaN,Gloria Swanson,Sadie Thompson


In [4]:
aa_data = aa_data.fillna('Nominee')
aa_data = aa_data.replace({'Winner':{1:'Winner'}})
aa_data = aa_data.rename(columns={'Winner':'Result'})
aa_data = aa_data.drop('Year',axis=1)

In [5]:
aa_data.head()

,Ceremony,Award,Result,Name,Film
0,1,Actor,Nominee,Richard Barthelmess,The Noose
1,1,Actor,Winner,Emil Jannings,The Last Command
2,1,Actress,Nominee,Louise Dresser,A Ship Comes In
3,1,Actress,Winner,Janet Gaynor,7th Heaven
4,1,Actress,Nominee,Gloria Swanson,Sadie Thompson


---

In [6]:
x = aa_data.to_json(path_or_buf='AA.json',orient='index')

In [7]:
pd.read_json('AA.json').transpose()

,Award,Ceremony,Film,Name,Result
0,Actor,1,The Noose,Richard Barthelmess,Nominee
1,Actor,1,The Last Command,Emil Jannings,Winner
10,Cinematography,1,Sunrise,Karl Struss,Winner
100,Outstanding Production,3,Universal,All Quiet on the Western Front,Winner
1000,Music (Original Score),13,Roy Webb,My Favorite Wife,Nominee
1001,Music (Original Score),13,Victor Young,North West Mounted Police,Nominee
1002,Music (Original Score),13,Werner Heymann,One Million B.C.,Nominee
1003,Music (Original Score),13,Aaron Copland,Our Town,Nominee
1004,Music (Original Score),13,"Leigh Harline, Paul J. Smith, Ned Washington",Pinocchio,Winner
1005,Music (Original Score),13,Franz Waxman,Rebecca,Nominee


In [8]:
meta = pd.read_csv('movies_metadata.csv')
kw = pd.read_csv('keywords.csv')
meta = meta[meta['id'].str.contains('-') == False]
meta['id'] = pd.to_numeric(meta['id'])

In [9]:
combined = kw.merge(meta,how='inner',on='id')

In [10]:
combined

,id,keywords,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,en,Toy Story,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,tt0113497,en,Jumanji,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,tt0113228,en,Grumpier Old Men,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,en,Waiting to Exhale,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0113041,en,Father of the Bride Part II,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,949,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '...",False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,tt0113277,en,Heat,...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,11860,"[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam...",False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0114319,en,Sabrina,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,45325,[],False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,tt0112302,en,Tom and Huck,...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,9091,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562...",False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,tt0114576,en,Sudden Death,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,710,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam...",False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,tt0113189,en,GoldenEye,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
